#🛠️ IFC Property Extractor

**Install dependencies**


In [2]:
!pip install gradio huggingface_hub ifcopenshell pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.7 MB/s eta 0:00:00


**Save dependencies to requirements.txt**

In [ ]:
%%writefile requirements.txt
gradio
ifcopenshell
pandas
openpyxl

Overwriting requirements.txt


**Create App**

In [3]:
# comment the line below inorder to debug the app here in colab
%%writefile app.py
import gradio as gr
import ifcopenshell
import pandas as pd
from tempfile import NamedTemporaryFile
import os

def extract_properties(ifc_file_path, element_class):
    try:
        model = ifcopenshell.open(ifc_file_path)

        results = []
        elements = model.by_type("IfcElement")
        # Get unique type names
        element_types = sorted(list(set(elem.is_a() for elem in elements)))
        classes_to_check = [element_class] if element_class != "" else sorted(list(set(obj.is_a() for obj in element_types)))

        for class_name in classes_to_check:
            elements = model.by_type(class_name)
            for element in elements:
                # Get all properties
                properties = {}
                for definition in element.IsDefinedBy:
                    if definition.is_a("IfcRelDefinesByProperties"):
                        prop_set = definition.RelatingPropertyDefinition
                        if prop_set.is_a("IfcPropertySet"):
                            for prop in prop_set.HasProperties:
                                properties[f"{prop_set.Name}.{prop.Name}"] = prop.NominalValue.wrappedValue if prop.NominalValue else None

                row = {
                    "GlobalId": element.GlobalId,
                    "Type": element.is_a(),
                    "Name": element.Name,
                    **properties
                }
                results.append(row)

        if not results:
            return pd.DataFrame({"Message": [f"No {element_class if element_class != '' else ''} elements found"]}), None, None

        df = pd.DataFrame(results)
        # Create temporary files for downloads
        with NamedTemporaryFile(delete=False, suffix='.csv') as tmp_csv:
            df.to_csv(tmp_csv.name, index=False)
            csv_path = tmp_csv.name

        with NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp_excel:
            df.to_excel(tmp_excel.name, index=False)
            excel_path = tmp_excel.name

        return df, csv_path, excel_path

    except Exception as e:
        return pd.DataFrame({"Error": [f"Processing failed: {str(e)}"]}), None, None

def extract_ifc_types(ifc_file):
    """Extract all IFC types from uploaded file"""
    if not ifc_file:
        return []
    try:
        ifc = ifcopenshell.open(ifc_file.name)
        # Get all elements that are IfcElement or its subtypes
        elements = ifc.by_type("IfcElement")
        # Get unique type names
        element_types = sorted(list(set(elem.is_a() for elem in elements)))
        return element_types
    except:
        return []

def update_dropdown(ifc_file):
    """Update dropdown options when file is uploaded"""
    choices = extract_ifc_types(ifc_file)
    return gr.Dropdown(choices=choices, value="", interactive=True)

# Gradio Interface
with gr.Blocks(title="IFC Property Extractor") as demo:
    gr.Markdown("## 🛠️ IFC Property Extractor")

    with gr.Row():
        ifc_input = gr.File(
            label="1. Upload IFC File",
            file_types=[".ifc"],
            type="filepath"
        )

    with gr.Row():
        with gr.Column():
            class_filter = gr.Dropdown(
                [""],
                value="",
                label="2. Filter by Class",
                interactive=False
            )
            extract_btn = gr.Button("Extract Properties", variant="primary")

        with gr.Column():
            csv_download = gr.File(label="Download CSV", visible=False)
            excel_download = gr.File(label="Download Excel", visible=False)

    output_table = gr.Dataframe(
        label="Extracted Properties",
        interactive=True,
        wrap=False,
    )

    # Update dropdown when file is uploaded
    ifc_input.change(
        fn=update_dropdown,
        inputs=ifc_input,
        outputs=[class_filter]
    )

    extract_btn.click(
        fn=extract_properties,
        inputs=[ifc_input, class_filter],
        outputs=[output_table, csv_download, excel_download]
    )

    # Show download buttons only when files are available
    extract_btn.click(
        lambda: [gr.File(visible=True), gr.File(visible=True)],
        outputs=[csv_download, excel_download]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4b52c4401bcb2a9a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Deployment Code**

In [ ]:

from huggingface_hub import HfApi, create_repo, upload_file
import os

os.environ["HF_TOKEN"] = "your_hugging_face_token"
api = HfApi(token=os.environ["HF_TOKEN"])

user_name = "your_user_name"
space_name = "your-space-name"

# Create a new Space (skip if already exists)
try:
    api.delete_repo(
        repo_id=f"{user_name}/{space_name}",
        repo_type="space",
        token=os.environ["HF_TOKEN"]
    )
    print("Deleted existing Space successfully")
except Exception as delete_error:
    print(f"Delete failed (may not exist): {str(delete_error)}")

api.create_repo(
    repo_id=f"{user_name}/{space_name}",
    repo_type="space",
    space_sdk="gradio",
    private=False,
)

# Upload app.py and requirements.txt
upload_file(
    path_or_fileobj="app.py",
    path_in_repo="app.py",
    repo_id=f"{user_name}/{space_name}",
    repo_type="space",
    token=os.environ["HF_TOKEN"]
)

upload_file(
    path_or_fileobj="requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=f"{user_name}/{space_name}",
    repo_type="space",
    token=os.environ["HF_TOKEN"]
)

print(f"Your app is live at: https://huggingface.co/spaces/{user_name}/{space_name}")